In [114]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, roc_auc_score

In [115]:
df=pd.read_csv('../churn_predictor.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [116]:
df.shape

(7043, 21)

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [118]:
df.describe(include='all')

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
count,7043,7043,7043.000000,7043,7043,7043.000000,7043,7043,7043,7043,...,7043,7043,7043,7043,7043,7043,7043,7043.000000,7043,7043
unique,7043,2,NaN,2,2,NaN,2,3,3,3,...,3,3,3,3,3,2,4,NaN,6531,2
top,7590-VHVEG,Male,NaN,No,No,NaN,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,20.2,No
freq,1,3555,NaN,3641,4933,NaN,6361,3390,3096,3498,...,3095,3473,2810,2785,3875,4171,2365,NaN,11,5174
mean,NaN,NaN,0.162147,NaN,NaN,32.371149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.761692,NaN,NaN
std,NaN,NaN,0.368612,NaN,NaN,24.559481,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.090047,NaN,NaN
min,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.250000,NaN,NaN
25%,NaN,NaN,0.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.500000,NaN,NaN
50%,NaN,NaN,0.000000,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.350000,NaN,NaN
75%,NaN,NaN,0.000000,NaN,NaN,55.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.850000,NaN,NaN


In [119]:
# Afficher les valeurs uniques pour la variable cible
print("\nDistribution de la variable cible (Attrition):")
print(df['Churn'].value_counts(normalize=True))


Distribution de la variable cible (Attrition):
Churn
No     0.73463
Yes    0.26537
Name: proportion, dtype: float64


In [120]:
# Convertir 'TotalCharges' en numérique, les erreurs (chaînes vides) deviennent NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

df['TotalCharges'].isnull().sum()

np.int64(11)

In [121]:
# Gérer les NaN (clients avec tenure=0). Remplaçons-les par 0 ou la médiane.
# Étant donné que ces clients ont tenure=0, TotalCharges devrait être 0.
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df['TotalCharges'].isnull().sum()

np.int64(0)

In [122]:
# Supprimer la colonne inutile (Customer ID)
df = df.drop('customerID', axis=1)
df.shape

(7043, 20)

In [123]:
# Encodage de la variable cible 'Churn'
df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})

C:\Users\poste\AppData\Local\Temp\ipykernel_16956\4028899368.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn'] = df['Churn'].replace({'Yes': 1, 'No': 0})


In [124]:
# Encodage des autres variables binaires (Oui/Non)
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    df[col] = df[col].replace({'Yes': 1, 'No': 0})

C:\Users\poste\AppData\Local\Temp\ipykernel_16956\2284107938.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].replace({'Yes': 1, 'No': 0})


In [125]:
# Gérer 'gender' (Homme/Femme)
df['gender'] = df['gender'].replace({'Female': 1, 'Male': 0})

C:\Users\poste\AppData\Local\Temp\ipykernel_16956\327999618.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['gender'] = df['gender'].replace({'Female': 1, 'Male': 0})


In [126]:
# Gérer 'SeniorCitizen' qui est déjà en (1, 0)
# Renommer 'SeniorCitizen' en 'Is_Senior' pour plus de clarté si 
df = df.rename(columns={'SeniorCitizen': 'Is_Senior'})

In [127]:
# Identifier les colonnes catégorielles restantes
categorical_cols = [
    'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport',
    'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod'
]

# Application de l'encodage One-Hot
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

print("\nNouvelles dimensions après One-Hot Encoding:", df_encoded.shape)
df_encoded.info()
df_encoded.isnull().sum().sum()


Nouvelles dimensions après One-Hot Encoding: (7043, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   gender                                 7043 non-null   int64  
 1   Is_Senior                              7043 non-null   int64  
 2   Partner                                7043 non-null   int64  
 3   Dependents                             7043 non-null   int64  
 4   tenure                                 7043 non-null   int64  
 5   PhoneService                           7043 non-null   int64  
 6   PaperlessBilling                       7043 non-null   int64  
 7   MonthlyCharges                         7043 non-null   float64
 8   TotalCharges                           7043 non-null   float64
 9   Churn                                  7043 non-null   int64  
 10  MultipleLines_N

np.int64(0)

In [128]:
# Supposons que df_encoded est le DataFrame après l'encodage One-Hot
# (Nous devons re-définir X et y et re-séparer, juste au cas où)

# Séparer les features (X) et la cible (y)
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Séparation en ensembles d'entraînement et de test (Stratification pour maintenir la proportion de Churn)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Définir les colonnes numériques
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

# 1. Instancier le StandardScaler
scaler = StandardScaler()

# 2. Entraîner le scaler sur l'ENSEMBLE D'ENTRAÎNEMENT et transformer
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])

# 3. Transformer l'ENSEMBLE DE TEST avec le scaler entraîné
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("Standardisation terminée :")
print(f"Moyennes de X_train normalisé pour {numerical_cols[0]}: {X_train[numerical_cols[0]].mean():.2f}")
print(f"Écart-type de X_train normalisé pour {numerical_cols[0]}: {X_train[numerical_cols[0]].std():.2f}")
print("\nLes ensembles X_train et X_test sont maintenant prêts pour la modélisation.")

Standardisation terminée :
Moyennes de X_train normalisé pour tenure: -0.00
Écart-type de X_train normalisé pour tenure: 1.00

Les ensembles X_train et X_test sont maintenant prêts pour la modélisation.


In [129]:
# Afficher la distribution de la classe cible avant SMOTE (dans l'ensemble d'entraînement)
print("Distribution de la classe avant SMOTE:", Counter(y_train))

# Instancier SMOTE
smote = SMOTE(random_state=42)

# Appliquer SMOTE uniquement sur les données d'entraînement
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Afficher la nouvelle distribution de la classe
print("Distribution de la classe après SMOTE:", Counter(y_train_resampled))

Distribution de la classe avant SMOTE: Counter({0: 4139, 1: 1495})
Distribution de la classe après SMOTE: Counter({0: 4139, 1: 4139})


In [130]:
# Fonction pour évaluer et afficher les métriques
def evaluate_model(y_true, y_pred, y_probs, model_name):
    """Calcule et affiche les métriques de performance pour un classifieur."""
    print(f"--- Résultats pour {model_name} ---")
    print(f"Accuracy (Précision globale): {accuracy_score(y_true, y_pred):.4f}")
    print(f"Recall (Rappel) : {recall_score(y_true, y_pred):.4f}")
    print(f"AUC Score: {roc_auc_score(y_true, y_probs):.4f}")
    print("Matrice de Confusion:\n", confusion_matrix(y_true, y_pred))

# ----------------------------------------------------
# 1. Régression Logistique (Modèle de Base)
# ----------------------------------------------------
lr_model = LogisticRegression(random_state=42, solver='liblinear')
# Entraînement sur les données rééquilibrées
lr_model.fit(X_train_resampled, y_train_resampled)

# Prédiction sur l'ensemble de TEST non vu
lr_preds = lr_model.predict(X_test)
lr_probs = lr_model.predict_proba(X_test)[:, 1]

evaluate_model(y_test, lr_preds, lr_probs, "Régression Logistique (avec SMOTE)")

print("\n" + "="*70 + "\n")

# ----------------------------------------------------
# 2. Random Forest (Modèle Robuste)
# ----------------------------------------------------
rf_model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=10, class_weight='balanced')
# NOTE: Le paramètre class_weight='balanced' du RF est souvent plus efficace que SMOTE
# Nous allons donc l'entraîner SANS les données SMOTE pour comparer
rf_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de TEST non vu
rf_preds = rf_model.predict(X_test)
rf_probs = rf_model.predict_proba(X_test)[:, 1]

evaluate_model(y_test, rf_preds, rf_probs, "Random Forest (avec pondération interne)")

--- Résultats pour Régression Logistique (avec SMOTE) ---
Accuracy (Précision globale): 0.7367
Recall (Rappel) : 0.7086
AUC Score: 0.8201
Matrice de Confusion:
 [[773 262]
 [109 265]]


--- Résultats pour Random Forest (avec pondération interne) ---
Accuracy (Précision globale): 0.7665
Recall (Rappel) : 0.7219
AUC Score: 0.8400
Matrice de Confusion:
 [[810 225]
 [104 270]]


In [131]:
# Nous allons utiliser le Random Forest pour l'importance des variables
feature_importances = pd.Series(
    rf_model.feature_importances_, 
    index=X_train.columns
).sort_values(ascending=False)

print("\n--- 10 Variables les Plus Importantes ---")
print(feature_importances.head(10))

# Visualisation (optionnel mais très utile)
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 6))
# feature_importances.head(10).plot(kind='barh')
# plt.title('Importance des variables pour la prédiction du Churn')
# plt.show()


--- 10 Variables les Plus Importantes ---
tenure                            0.177895
TotalCharges                      0.147359
MonthlyCharges                    0.100018
Contract_Two year                 0.090402
InternetService_Fiber optic       0.067879
PaymentMethod_Electronic check    0.050874
Contract_One year                 0.038687
OnlineSecurity_Yes                0.033176
TechSupport_Yes                   0.025971
PaperlessBilling                  0.020377
dtype: float64


In [132]:
# Définition de la métrique d'évaluation
# Nous utilisons 'roc_auc' pour optimiser notre modèle sur cette métrique
auc_scorer = make_scorer(roc_auc_score)

# Définition de la grille de paramètres à tester
param_grid = {
    # Nombre d'arbres dans la forêt
    'n_estimators': [100, 200, 300],
    # Profondeur maximale de l'arbre
    'max_depth': [5, 10, 15],
    # Nombre minimum d'échantillons requis pour diviser un nœud interne
    'min_samples_split': [5, 10]
}

# Instanciation du modèle avec le poids des classes pour gérer le déséquilibre
rf_base = RandomForestClassifier(random_state=42, class_weight='balanced')

# Instanciation de la recherche par grille
grid_search = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid,
    scoring=auc_scorer,  # Optimiser l'AUC
    cv=3,                # Utiliser 3-fold Cross-Validation
    verbose=1,           # Afficher la progression
    n_jobs=1           # Utiliser un seul cœur du processeur
)

# Entraînement de la recherche par grille (cette étape est la plus longue)
grid_search.fit(X_train, y_train) 

# Afficher les meilleurs résultats
print("\n--- Résultats de la Grid Search ---")
print(f"Meilleurs Hyperparamètres trouvés: {grid_search.best_params_}")
print(f"Meilleur score AUC (Validation Croisée): {grid_search.best_score_:.4f}")

# Sauvegarder le meilleur modèle
best_rf_model = grid_search.best_estimator_

# Évaluation finale sur l'ensemble de test
final_preds = best_rf_model.predict(X_test)
final_probs = best_rf_model.predict_proba(X_test)[:, 1]

print("\n--- Évaluation Finale du Meilleur Modèle ---")
evaluate_model(y_test, final_preds, final_probs, "Random Forest Optimisé")

Fitting 3 folds for each of 18 candidates, totalling 54 fits

--- Résultats de la Grid Search ---
Meilleurs Hyperparamètres trouvés: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
Meilleur score AUC (Validation Croisée): 0.7663

--- Évaluation Finale du Meilleur Modèle ---
--- Résultats pour Random Forest Optimisé ---
Accuracy (Précision globale): 0.7388
Recall (Rappel) : 0.8182
AUC Score: 0.8408
Matrice de Confusion:
 [[735 300]
 [ 68 306]]


In [133]:
# 1. Enregistrer le modèle optimisé
joblib.dump(best_rf_model, 'best_churn_model.pkl')

# 2. Enregistrer l'objet StandardScaler (pour mettre à l'échelle les nouvelles données d'entrée)
# Note : L'objet 'scaler' a été créé à l'étape 2.B.
joblib.dump(scaler, 'scaler.pkl')

# 3. Enregistrer la liste ordonnée des colonnes (pour garantir l'ordre des features de l'API)
joblib.dump(X_train.columns.tolist(), 'model_columns.pkl')

print("\nModèle, Scaler et Ordre des Colonnes enregistrés avec succès dans des fichiers .pkl.")
print("Vous avez tous les artefacts nécessaires pour le déploiement!")


Modèle, Scaler et Ordre des Colonnes enregistrés avec succès dans des fichiers .pkl.
Vous avez tous les artefacts nécessaires pour le déploiement!
